### 1. Importing libraries

In [9]:
import os
import os.path
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime
import plotly.express as px
import plotly.graph_objects as go
import base64

### 2. Importing data

In [ ]:
# Define the directory and file path for the dataset
datadir = "data/tidy_data"
# Load the dataset into a DataFrame
data = pd.read_csv(os.path.join(datadir, "new_jump_data.csv"))

# Create a binary variable 'injury_log' to indicate injury occurrence:
# 1 for rows where 'injury_type' is not NaN (indicating an injury), and 0 otherwise
data["injury_log"] = data["injury_type"].notna().astype(int)

marginal_df = data[[
    "athlete_code", "Date", "mRSI", "Peak Braking Velocity", "Peak Propulsive Force", "Avg. Braking Power", "Peak Braking Force", "injury_log"
]]

def date_coversion(date):
    # Check if the date is a valid string before processing
    if isinstance(date, str):
        try:
            date_object = datetime.strptime(date, '%m-%d-%Y')
            iso_format_date = date_object.strftime('%Y-%m-%d')
            return iso_format_date
        except ValueError:
            return None  # Return None if the date format is incorrect
    return date  # If it's not a string, return the value as is (e.g., NaN or float)

# Changing date formatting
marginal_df["Date"] = marginal_df["Date"].apply(date_coversion)# Convert to datetime format (this will handle invalid or missing dates gracefully)
marginal_df["Date"] = pd.to_datetime(marginal_df["Date"])

### 3. Aggregating data

In [ ]:
marginal_df = data[[
    "athlete_code", "Date", "mRSI", "Peak Braking Velocity", "Peak Propulsive Force", "Avg. Braking Power", "Peak Braking Force", "injury_log"
]]

marginal_data = (
    marginal_df.groupby(['athlete_code', 'Date'], as_index=False)
    .agg(
        Avg_mRSI=('mRSI', 'mean'),
        Avg_Peak_Braking_Velocity=('Peak Braking Velocity', 'mean'),
        Avg_Peak_Propulsive_Force=('Peak Propulsive Force', 'mean'),
        Avg_Avg_Braking_Power=('Avg. Braking Power', 'mean'),
        Avg_Peak_Braking_Force=('Peak Braking Force', 'mean')
    )
)

# Calculate the percentage change for each column (mRSI and others)
marginal_data['mRSI'] = marginal_data['Avg_mRSI'].pct_change()
marginal_data['Peak_Braking_Velocity'] = marginal_data['Avg_Peak_Braking_Velocity'].pct_change()
marginal_data['Peak_Propulsive_Force'] = marginal_data['Avg_Peak_Propulsive_Force'].pct_change()
marginal_data['Avg_Braking_Power'] = marginal_data['Avg_Avg_Braking_Power'].pct_change()
marginal_data['Peak_Braking_Force'] = marginal_data['Avg_Peak_Braking_Force'].pct_change()

marginal_data["Date"] = marginal_data["Date"].apply(date_coversion)# Convert to datetime format (this will handle invalid or missing dates gracefully)
marginal_data["Date"] = pd.to_datetime(marginal_data["Date"])

### 4. Creating the interactive plot

In [11]:
# Function to create the plot for a specific athlete and selected metric
def create_plot(athlete_code, metric='mRSI_marginal_change', filename="athlete_plot.html"):
    # Filter data for the selected athlete
    marginal_data_single = marginal_data[marginal_data['athlete_code'] == athlete_code]

    # Create Plotly figure
    fig = go.Figure()

    # Add line plot (black line)
    fig.add_trace(go.Scatter(
        x=marginal_data_single['Date'],
        y=marginal_data_single[metric],
        mode='lines',
        line=dict(color='black'),
        name='Line'
    ))

    # Add scatter plot (red points)
    fig.add_trace(go.Scatter(
        x=marginal_data_single['Date'],
        y=marginal_data_single[metric],
        mode='markers',
        marker=dict(color='red', size=8),
        name='Points'
    ))

    # Add logo image (optional)
    with open("logo/horizontal_logo.png", "rb") as img:
        encoded_image = base64.b64encode(img.read()).decode()

    fig.add_layout_image(
        dict(
            source=f"data:image/png;base64,{encoded_image}",
            xref="paper", yref="paper",
            x=0.99, y=1.05,
            sizex=0.15, sizey=0.15,
            xanchor="right", yanchor="bottom"
        )
    )

    # Add title and labels
    fig.update_layout(
        font_family="Helvetica",
        width=1600,
        height=900,
        title=dict(
            text=f"<b>Marginal Percentage Change in {metric.replace('_', ' ').title()} for Athlete {athlete_code}</b>",
            font=dict(size=28, family="Helvetica"),
            x=0.018,
            y = 0.93
        ),
        xaxis_title="Date",
        yaxis_title=f"Marginal Percentage Change in {metric.replace('_', ' ').title()}",
        xaxis=dict(tickangle=45),
        annotations=[
            dict(
                x=-0.037,
                y=1.065,
                text="Choose metric here:",
                showarrow=False,
                font=dict(size=20, family="Helvetica"),
                xref="paper",
                yref="paper",
                align="left"
            ),
            dict(
                x=0.37,
                y=1.065,
                text="Choose athlete here:",
                showarrow=False,
                font=dict(size=20, family="Helvetica"),
                xref="paper",
                yref="paper",
                align="left"
            )
        ],
        template="plotly_white",
        updatemenus=[
            dict(
                buttons=[
                    # Add buttons for each athlete
                    dict(
                        args=[{"x": [marginal_data[marginal_data['athlete_code'] == athlete]['Date']],
                               "y": [marginal_data[marginal_data['athlete_code'] == athlete][metric]]},
                              {"title": f"<b>Marginal Percentage Change in {metric.replace('_', ' ').title()} for Athlete {athlete}</b>"}],
                        label=athlete,
                        method="update"
                    ) for athlete in marginal_data['athlete_code'].unique()
                ],
                direction="down",
                showactive=True,
                x=0.442,
                xanchor="left",
                y=1.15,
                yanchor="top",
                pad={"t": 50}  # Space for the dropdown
            ),
            # Dropdown for metric selection
            dict(
                buttons=[
                    dict(
                        args=[{"y": [marginal_data[marginal_data['athlete_code'] == athlete_code][metric]]},
                              {"title": f"<b>Marginal Percentage Change in {metric.replace('_', ' ').title()} for Athlete {athlete_code}</b>"}],
                        label=metric.replace('_', ' ').title(),
                        method="update"
                    ) for metric in [
                        'mRSI',
                        'Peak_Braking_Velocity',
                        'Peak_Propulsive_Force',
                        'Avg_Braking_Power',
                        'Peak_Braking_Force'
                    ]
                ],
                direction="down",
                showactive=True,
                x=0.1,
                xanchor="left",
                y=1.225,
                yanchor="top",
                pad={"t": 100}  # Space for the dropdown
            )
        ]
    )

    # Save the plot as an HTML file
    fig.write_html(filename)

    # Show the plot
    fig.show()

# List of available athlete codes
athlete_codes = marginal_data['athlete_code'].unique()

# Create the initial plot for the first athlete and default metric
create_plot(athlete_codes[0], metric='mRSI', filename="test_athlete_plot.html")
